#### Bootstrap OpenAssetIO

In [12]:
import json
import pprint

from openassetio import BatchElementException
from openassetio.hostApi import HostInterface, ManagerFactory
from openassetio.log import ConsoleLogger, SeverityFilter
from openassetio.pluginSystem import PythonPluginSystemManagerImplementationFactory

class DemoHostInterface(HostInterface):
    def identifier(self):
        return "org.demo.jupyter"

    def displayName(self):
        return "Demo Host"


logger = SeverityFilter(ConsoleLogger())
impl_factory = PythonPluginSystemManagerImplementationFactory(logger)
host_interface = DemoHostInterface()

manager = ManagerFactory.defaultManagerForInterface("resources/config.toml", host_interface, impl_factory, logger)
context = manager.createContext()

def print_traits_data(data):
    as_dict = {
        trait_id: {
            property_key: data.getTraitProperty(trait_id, property_key)
            for property_key in data.traitPropertyKeys(trait_id)
        }
        for trait_id in data.traitSet()
    }
    print(json.dumps(as_dict, indent=4, sort_keys=True))

## Query an entity's version

Note the change in signature. We no longer modify the `context` to set an `access` mode, it is instead a workflow-specific enum that is a first-class argument of the `resolve` function.

BAL now supports resolving the `VersionTrait`.

Oh no, the display name has a spelling mistake!

In [3]:
from openassetio.access import ResolveAccess

from openassetio_mediacreation.traits.lifecycle import VersionTrait
from openassetio_mediacreation.traits.identity import DisplayNameTrait

latest_ref = manager.createEntityReference("bal:///demo_project/logos/openassetio")

print(latest_ref)
data = manager.resolve(latest_ref, {DisplayNameTrait.kId, VersionTrait.kId}, ResolveAccess.kRead, context)
print_traits_data(data)

bal:///demo_project/logos/openassetio
{
    "openassetio-mediacreation:identity.DisplayName": {
        "name": "The OpenAssetOI logo",
        "qualifiedName": "logos / openassetio"
    },
    "openassetio-mediacreation:lifecycle.Version": {
        "specifiedTag": "latest",
        "stableTag": "1"
    }
}


## Get all versions (including metaversions)

BAL now supports relationship queries for the `EntityVersionsRelationshipSpecification`. This retrieves all versions, including metaversions such as "latest".

(Relationship queries look a bit clunky since we currently only expose a batch-first callback-based API)

In [5]:
from openassetio.access import RelationsAccess

from openassetio_mediacreation.specifications.lifecycle import EntityVersionsRelationshipSpecification

related_refs = []

relationship = EntityVersionsRelationshipSpecification.create().traitsData()

manager.getWithRelationship(
    [latest_ref], relationship, RelationsAccess.kRead, context, 
    lambda _, refs: related_refs.extend(refs), lambda _, err: print(err.message))

print([str(ref) for ref in related_refs])

['bal:///demo_project/logos/openassetio', 'bal:///demo_project/logos/openassetio?v=1']


## Query a specific version

"v=1" and "latest" are the same entity, but different results are returned for their "stableTag".

In [6]:
assert latest_ref == related_refs[0]
v1_ref = related_refs[1]

print(v1_ref)
data = manager.resolve(v1_ref, {DisplayNameTrait.kId, VersionTrait.kId}, ResolveAccess.kRead, context)
print_traits_data(data)

print(latest_ref)
data = manager.resolve(latest_ref, {DisplayNameTrait.kId, VersionTrait.kId}, ResolveAccess.kRead, context)
print_traits_data(data)

bal:///demo_project/logos/openassetio?v=1
{
    "openassetio-mediacreation:identity.DisplayName": {
        "name": "The OpenAssetOI logo",
        "qualifiedName": "logos / openassetio"
    },
    "openassetio-mediacreation:lifecycle.Version": {
        "specifiedTag": "1",
        "stableTag": "1"
    }
}
bal:///demo_project/logos/openassetio
{
    "openassetio-mediacreation:identity.DisplayName": {
        "name": "The OpenAssetOI logo",
        "qualifiedName": "logos / openassetio"
    },
    "openassetio-mediacreation:lifecycle.Version": {
        "specifiedTag": "latest",
        "stableTag": "1"
    }
}


## Publish a new version

Let's fix the spelling mistake.

Nothing new here, other than the final reference returned from `register` has the version encoded in it.

In [7]:
from openassetio import TraitsData
from openassetio.access import PublishingAccess

new_data = TraitsData(data)
display_name_trait = DisplayNameTrait(new_data)
display_name_trait.imbue()

working_ref = manager.preflight(latest_ref, new_data, PublishingAccess.kWrite, context)

display_name_trait.setName("The OpenAssetIO logo")

new_ref = manager.register(latest_ref, new_data, PublishingAccess.kWrite, context)
print(new_ref)

bal:///demo_project/logos/openassetio?v=2


## Check it's updated

Let's compare the data associated with all the references we know of.

Now, "v=2" and "latest" are the same entity.  "v=1" remains unmodified.

In [9]:
print(v1_ref)
data = manager.resolve(v1_ref, {DisplayNameTrait.kId, VersionTrait.kId}, ResolveAccess.kRead, context)
print_traits_data(data)

print(new_ref)
data = manager.resolve(new_ref, {DisplayNameTrait.kId, VersionTrait.kId}, ResolveAccess.kRead, context)
print_traits_data(data)

print(latest_ref)
data = manager.resolve(latest_ref, {DisplayNameTrait.kId, VersionTrait.kId}, ResolveAccess.kRead, context)
print_traits_data(data)

bal:///demo_project/logos/openassetio?v=1
{
    "openassetio-mediacreation:identity.DisplayName": {
        "name": "The OpenAssetOI logo",
        "qualifiedName": "logos / openassetio"
    },
    "openassetio-mediacreation:lifecycle.Version": {
        "specifiedTag": "1",
        "stableTag": "1"
    }
}
bal:///demo_project/logos/openassetio?v=2
{
    "openassetio-mediacreation:identity.DisplayName": {
        "name": "The OpenAssetIO logo",
        "qualifiedName": "logos / openassetio"
    },
    "openassetio-mediacreation:lifecycle.Version": {
        "specifiedTag": "2",
        "stableTag": "2"
    }
}
bal:///demo_project/logos/openassetio
{
    "openassetio-mediacreation:identity.DisplayName": {
        "name": "The OpenAssetIO logo",
        "qualifiedName": "logos / openassetio"
    },
    "openassetio-mediacreation:lifecycle.Version": {
        "specifiedTag": "latest",
        "stableTag": "2"
    }
}


## Get all versions (including metaversions)

Let's check what versions are now available.  

Here we use `latest_ref`, but we could use one of the versioned refs and the results would be the same.

In [10]:
related_refs = []

relationship = EntityVersionsRelationshipSpecification.create().traitsData()

manager.getWithRelationship(
    [latest_ref], relationship, RelationsAccess.kRead, context, 
    lambda _, refs: related_refs.extend(refs), lambda _, err: print(err.message))

print([str(ref) for ref in related_refs])

['bal:///demo_project/logos/openassetio', 'bal:///demo_project/logos/openassetio?v=2', 'bal:///demo_project/logos/openassetio?v=1']


## Get all versions (excluding metaversions)

BAL also now supports the `StableEntityVersionsRelationshipSpecification` to query for entity versions excluding metaversions like "latest".

In [11]:
from openassetio_mediacreation.specifications.lifecycle import StableEntityVersionsRelationshipSpecification


related_refs = []

relationship = StableEntityVersionsRelationshipSpecification.create().traitsData()

manager.getWithRelationship(
    [latest_ref], relationship, RelationsAccess.kRead, context, 
    lambda _, refs: related_refs.extend(refs), lambda _, err: print(err.message))

print([str(ref) for ref in related_refs])

['bal:///demo_project/logos/openassetio?v=2', 'bal:///demo_project/logos/openassetio?v=1']
